For this lab, you will be using the `trip_fare_500k.csv` file found in the `data/nycTaxiData/` folder .

This dataset contains a fairly large number of distinct trips taken in cabs in the NYC area in 2013 (500 thousand of them, to be exact!).

The dataset contains the following information at the top of the file (this is called the header):

* `medallion`: The ID of the cab being operated
* `hack_license`: The ID of the person operating the cab
* `vendor_id`: The type of vendor operating the cab, can either be `CMT` or `VTS`, no clue what these two types mean
* `pickup_datetime`: The time when the ride started
* `payment_type`: How the trip was paid, `UNK` stands for unknown, I have no idea what `NOC` stands for, but lets assume its some known way to pay
* `fare_amount`: Base fare cost of the trip
* `surcharge`: Additional charges that are not tolls
* `mta_tax`: The mta has to get its cut, right? :)
* `tip_amount`: How generous the rider(s) decided to be
* `tolls_amount`: How much money you had to pay in tolls
* `total_amount`: How much the trip cost, all in

Phew, That's a lot of info!

Here is the assignment:

1. What was the most expensive/least expensive trip taken?
* Does the overall `total_amount` paid per ride correlate with `tip_amount` per ride?
* Does it correlate when you remove all rides with unknown `payment_type`?
* Calculate the average cost of a trip in this dataset given the following conditions:
  1. Across the whole dataset
  2. Across the whole dataset when the `payment_type` is known (not `UNK`)
  3. For each `payment_type`
  4. Which `payment_type` had the highest average cost?
  5. Which `payment_type` had the largest spread in how much people paid (largest standard deviation)?
  6. Which `payment_type` had the most generous people (had the highest average tip), including unknown payment types?
  7. What hour in the day were people most generous, on average, when they got into a cab?
  8. What hour of the day did people fluctuate the most in terms of tips? That is, do some hours lead to unpredictable tip amounts? 
* Which person (`hack_license`) made the most money:
  1. In total
  2. On a per-trip basis, given that they took at least 20 trips
* Does the number of trips a given cabbie takes (her/his experience) correlate with how well she/he is tipped? If so, in what direction?
* Does the number of times a given cab is used correlate with how well the person driving the cab is tipped? That is, are there "lucky" cabs?
* Which `vendor_id` had the higher average `surcharge` on a per-hour basis?
* Which hour in the day: 
  1. Did people most frequently take rides?
  2. Did people least frequently take rides?
  3. Had the largest number of unique cabs on the street?
  4. Had the least number of cabs in the street?
  5. What is the average number of cabs on the streets in NYC in each quarter of the day (at least in this dataset?)?

Use the rest of this notebook to work through all these questions. 

I will be coming around to everyone to help/guide you in your data science quest!

If you can tackle all of these questions, then you've learned a lot already! 

If not, don't worry, this stuff is hard and I will gladly help/guide you through all of this.

but take charge of your learning!

This means:

* Ask a neighbor to help if you don't understand something. 
* If your neighbor can't help you, try using:
  * the interactive documentation I showed you how to use earlier
  * [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/index.html)
  * [google](http://www.google.com)
  * [stackoverflow](http://stackoverflow.com) to see if someone in the internet ether has had a similar problem before
  * if none of this works, then I will gladly help you
* This will accomplish at least two things:
  * It will get you to use online resources and take charge of your learning
  * Get you to learn alternative approaches (those I did not show you today) to solving your problem

I've started the bare-bones script for you by:

* importing what I ~~think~~ know you will need
* loading the dataset into a variable called `fareData` that stores the data as a `DataFrame` object (you might need to change the path to where the file is located on your system)
* formatting the timestamp for you so that you don't have to figure out how to do it, because spending 30+ minutes  (or more) trying to figure it out is not the point of the assignment. This way, all of the functions in `fareData.pickup_datetime.dt` can immediately be used on the `pickup_datetime` column your dataset.

The rest I leave to you. Happy hacking!

In [1]:
import pandas as pd
import numpy as np

In [2]:
fareData = pd.read_csv("nycTaxiData/nycTaxiData/trip_fare_500k.csv")
fareData.pickup_datetime = pd.to_datetime(fareData.pickup_datetime,format="%Y-%m-%d %H:%M:%S")
fareData.dtypes #this is to confirm that the pickup_datetime column, as well as all of the other
# columns, are in the appropriate formats (pickup_datetime should be in datetime64 format)
# if it isn't something is wrong, and we need to figure what that is

medallion                  object
hack_license               object
vendor_id                  object
pickup_datetime    datetime64[ns]
payment_type               object
fare_amount               float64
surcharge                 float64
mta_tax                   float64
tip_amount                float64
tolls_amount              float64
total_amount              float64
dtype: object

In [3]:
# 1. What is the most expensive/least expensive trip taken?

mostexpensivefare = fareData['fare_amount'].max()
leastexpensivefare = fareData['fare_amount'].min()

print(mostexpensivefare)
print(leastexpensivefare)

460.0
2.5


In [8]:
# 2. Does the overall total_amount paid per ride correlate with tip_amount per ride? YES - 0.6709 shows positive correlation between the 2

correlationmatrix = fareData.corr()
print(correlationmatrix)

correlation = fareData.total_amount.corr(fareData.tip_amount)
print(correlation)

              fare_amount  surcharge   mta_tax  tip_amount  tolls_amount  \
fare_amount      1.000000  -0.064229 -0.256318    0.549345      0.614237   
surcharge       -0.064229   1.000000  0.023950   -0.021421     -0.063364   
mta_tax         -0.256318   0.023950  1.000000   -0.134473     -0.260192   
tip_amount       0.549345  -0.021421 -0.134473    1.000000      0.413718   
tolls_amount     0.614237  -0.063364 -0.260192    0.413718      1.000000   
total_amount     0.984733  -0.035148 -0.258903    0.670875      0.676342   

              total_amount  
fare_amount       0.984733  
surcharge        -0.035148  
mta_tax          -0.258903  
tip_amount        0.670875  
tolls_amount      0.676342  
total_amount      1.000000  
0.670875307140199


In [12]:
# 3. Does it correlate when you remove all rides with unknown payment type? YES - still a positive 0.6703 correlation

MaskUNK = fareData.payment_type != 'UNK'
KnownPaymentsData = fareData[MaskUNK]

corrKnownPayments = KnownPaymentsData.total_amount.corr(fareData.tip_amount)
print(corrKnownPayments)

0.6703435782941248


In [109]:
# 4. Calculate the average cost of a trip in this dataset given the following conditions:
# A. Across the whole dataset.
# B. Across the whole dataset when the payment is known (not UNK).
# C. For each payment_type.
# D. Which payment type has the highest average cost?
# E. Which payment type has the largest spread in how much people paid (standard deviation)?
# F. Which payment type has the most generous people, (has the highest average tip), including unknown payment types?
# G. What hour of the day were people most generous, on average, when they got in a cab?
# H. What hour of the day did people fluctuate most in terms of tips? That is, do some hours lead to unpredictable tip amounts?

avgcost = fareData.total_amount.mean()
print('A: ', avgcost)

avgcostpayknown = KnownPaymentsData.total_amount.mean()
print('B: ', avgcostpayknown)

paymentsgroup = fareData.groupby('payment_type')
avgcostbypayment = paymentsgroup.total_amount.mean()
print('C: ', avgcostbypayment)

maxavgcostpaymenttype = avgcostbypayment[avgcostbypayment == avgcostbypayment.max()]
print('D: ', maxavgcostpaymenttype)

stdcostbypayment = paymentsgroup.total_amount.std()
maxstdbypaymenttype = stdcostbypayment[stdcostbypayment == stdcostbypayment.max()]
print('E: ', maxstdbypaymenttype)

avgtipsbypayment = paymentsgroup.tip_amount.mean()
mostgeneroustype = avgtipsbypayment[avgtipsbypayment == avgtipsbypayment.max()]
print('F: ', mostgeneroustype)

fareData['hour'] = fareData.pickup_datetime.dt.hour
hourgroup = fareData.groupby('hour')
avgtipbyhour = hourgroup.tip_amount.mean()
mostgeneroushour = avgtipbyhour[avgtipbyhour == avgtipbyhour.max()]
print('G: ', mostgeneroushour)

stdtipbyhour = hourgroup.tip_amount.std()
largeststdhour = stdtipbyhour[stdtipbyhour == stdtipbyhour.max()]
print('H: ', largeststdhour)

A:  14.170294439997486
B:  14.163024804668963
C:  payment_type
CRD    16.371267
CSH    11.616747
DIS     5.750000
NOC     3.000000
UNK    22.116674
Name: total_amount, dtype: float64
D:  payment_type
UNK    22.116674
Name: total_amount, dtype: float64
E:  payment_type
UNK    21.049277
Name: total_amount, dtype: float64
F:  payment_type
UNK    3.430088
Name: tip_amount, dtype: float64
G:  hour
5    1.942813
Name: tip_amount, dtype: float64
H:  hour
5    3.275576
Name: tip_amount, dtype: float64


In [111]:
# 5. Which person (hack_license) made the most money:
# A. In total?
# B. On a per trip basis, given that they took at least 20 trips?

drivergroup = fareData.groupby('hack_license')
drivertotals = drivergroup.total_amount.sum()
mostmoneydriver = drivertotals[drivertotals == drivertotals.max()]
print('A: ', mostmoneydriver)

filtereddrivers = drivergroup.filter(lambda x: x.fare_amount.size >= 20)
filtereddrivergroup = filtereddrivers.groupby('hack_license')


avgfare = filtereddrivergroup.total_amount.mean()
mostmoneydriveravg = avgfare[avgfare == avgfare.max()]
print('B: ', mostmoneydriveravg)

A:  hack_license
CFCD208495D565EF66E7DFF9F98764DA    2517.28
Name: total_amount, dtype: float64
B:  hack_license
5114DF85775775ED4F53235D8478E80B    53.466667
Name: total_amount, dtype: float64


In [112]:
# 6. Does the number of trips a given cabbie takes (her/his experience) correlate with how well she/he is tipped? If so, in what direction?

# tripcountsperdriver = drivergroup.fare_amount.size()
# tripcounts = pd.DataFrame({'hack_license':tripcountsperdriver.index, 'total_trips':tripcountsperdriver.values})

# joinedtripcounts = tripcounts.merge(fareData,on='hack_license')

# tiptripcorr = joinedtripcounts.tip_amount.corr(joinedtripcounts.total_trips)
# print(tiptripcorr)

drivergrouptipvstrips = fareData.groupby("hack_license")["tip_amount"].agg([np.size, np.mean])
print(drivergrouptipvstrips.corr())

# a slight negative correlation, the more experience the cabbie has, the lower the tip. Not a strong correlation though

          size      mean
size  1.000000 -0.291082
mean -0.291082  1.000000


In [113]:
# 7. Does the number of times a given cab is used correlate with how well the person driving the cab is tipped? That is, are there "lucky" cabs?

# medalliongroup = fareData.groupby('medallion')
# medallionuses = medalliongroup.fare_amount.size()

# countsdataframe = pd.DataFrame({'medallion':medallionuses.index, 'total_uses':medallionuses.values})

# joinedcabusetotals = countsdataframe.merge(fareData,on='medallion')

# tipcabcorr = joinedcabusetotals.tip_amount.corr(joinedcabusetotals.total_uses)
# print(tipcabcorr)

taxigrouptipsvstrips = fareData.groupby("medallion")["tip_amount"].agg([np.size, np.mean])
print(taxigrouptipsvstrips.corr())

# again, only a slight negative correlation, the more times a cab is used, the lower the tip. Not a strong correlation.

         size     mean
size  1.00000 -0.34822
mean -0.34822  1.00000


In [114]:
# 8. Which vendor_id had the highest average surcharge on a per hour basis?

# vendorhourgroup = fareData.groupby(['vendor_id','hour'])
# avgsurcharge = vendorhourgroup.surcharge.mean()

# maxavgsurchargeperhour = avgsurcharge.groupby(level=1).idxmax()

# print(avgsurcharge)
# print(maxavgsurchargeperhour)

perVendorMeans = fareData.groupby(["vendor_id","hour"])["surcharge"].mean().unstack(level=1)
perVendorMeans = perVendorMeans.mean(axis=1)
print("The vendor with the highest average surcharge per-hour is:")
print(perVendorMeans[perVendorMeans==perVendorMeans.max()])


The vendor with the highest average surcharge per-hour is:
vendor_id
VTS    0.293235
dtype: float64


In [115]:
# 9. Which hour in the day:
# A. Did people most frequently take rides?
# B. Did people least frequently take rides?
# C. Had the largest number of unique cabs on the street?
# D. Had the least number of cabs on the street?
# E. What is the average number of cabs on the streets in NYC in each quarter of the day (at least in this dataset)?

hourgroup = fareData.groupby('hour')
ridetotals = hourgroup.fare_amount.size()

maxrideshour = ridetotals[ridetotals == ridetotals.max()]
print('A: ', maxrideshour)

minrideshour = ridetotals[ridetotals == ridetotals.min()]
print('B: ', minrideshour)

numuniquecabs = hourgroup.medallion.nunique()
mostcabshour = numuniquecabs[numuniquecabs == numuniquecabs.max()]
print('C: ', mostcabshour)

leastcabshour = numuniquecabs[numuniquecabs == numuniquecabs.min()]
print('D: ', leastcabshour)

fareData['quarterDay'] = pd.cut(fareData['hour'], [-1,5,11,17,np.inf])
earliestDay = fareData.pickup_datetime.dt.dayofyear.min()
lastDay = fareData.pickup_datetime.dt.dayofyear.max()
totalQuarters = (lastDay-earliestDay)*4
uniqueCabsQuarterDay = fareData.drop_duplicates(["medallion","quarterDay"])
print('E: ', uniqueCabsQuarterDay.groupby("quarterDay").size()/totalQuarters)

A:  hour
12    33564
Name: fare_amount, dtype: int64
B:  hour
4    3962
Name: fare_amount, dtype: int64
C:  hour
14    6201
Name: medallion, dtype: int64
D:  hour
5    1935
Name: medallion, dtype: int64
E:  quarterDay
(-1.0, 5.0]     73.460526
(5.0, 11.0]     86.302632
(11.0, 17.0]    92.434211
(17.0, inf]     91.552632
dtype: float64
